# Notebook Contents

- [Imports](#Imports)
- [Data](#Data)
- [Data Cleaning](#Data-Cleaning)
- [Preprocessing](#Preprocessing)
    - [Multicolinearity - VIF](#Multicolinearity---VIF)
- [Features](#Features)
- [Random Forest Modeling](#Random-Forest-Modeling)
    - [4-Seam](#Linear-Regression---4-Seam)
    - [Cutter](#Linear-Regression---Cutter)
    - [Sinker](#Linear-Regression---Sinker)
    - [Slider](#Linear-Regression---Slider)
    - [Curveball](#Linear-Regression---Curveball)
    - [Changeup](#Linear-Regression---Changeup)

# Imports

In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, k_means
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

import warnings
warnings.filterwarnings('ignore')

# Data

In [ ]:
runs = data[['home_score', 'away_score', 'post_home_score', 'post_away_score', 'home_runs', 'away_runs']]
runs.tail(10)

In [ ]:
re.groupby(['re']).mean()

# Data Cleaning

HR, 3B, 2B, 1B, reach on error, HBP, BB, IBB, out, K, passed ball, WP, balk, SB, CS, pickoff, pickoff error, OtherAdvance, interference, foulE?, DefensiveIndiff

In [ ]:
data.events.value_counts()

In [ ]:
data.description.value_counts()

# Preprocessing

### Multicolinearity - VIF

Velocity, Spin Rate, HB, VB, Release Extension, Horizontal Release Position, Vertical Release Position, Horizontal Plate Coords, Vertical Plate Coords

In [ ]:
#features = data[['velo', 'spin_rate', 'pfx_-x', 'pfx_z', 'release_extension', 
#                 'release_pos_x', 'release_pos_z', 'plate_x', 'plate_z',
#                 'pitch_type', 'p_throws']]
#features_vif = features.select_dtypes([np.number])
#vif_data = pd.DataFrame()
#vif_data["feature"] = features_vif.columns
#
#vif_data["VIF"] = [variance_inflation_factor(features_vif.values, i)
#                   for i in range(len(features_vif.columns))]
#
#vif_data.sort_values(by = 'VIF').head(10)

# Features

**Independent Variables:** Velocity, Spin Rate, VB, HB, Release Extension, Horizontal Release Position, Vertical Release Position, Horizontal Plate Coords, Vertical Plate Coords

**Dependent Variable:** re_play

Pitch Types:

Fastball: 4-Seam, Cutter, Sinker (FF, FC, SI)

Breaking Ball: Slider, Curveball, Knuckle Curve (SL, CU, KC)

Offspeed: Changeup, Splitter (CH, FS)

In [ ]:
ff = data.loc[data['pitch_type'] == 'FF']
fc = data.loc[data['pitch_type'] == 'FC']
fastball = ff.append(fc)
si = data.loc[data['pitch_type'] == 'SI']
fastball = fastball.append(si)
print('Fastball shape:', fastball.shape)
sl = data.loc[data['pitch_type'] == 'SL']
cu = data.loc[data['pitch_type'] == 'CU']
breaking_ball = sl.append(cu)
kc = data.loc[data['pitch_type'] == 'KC']
breaking_ball = breaking_ball.append(kc)
print('Breaking Ball:', breaking_ball.shape)
ch = data.loc[data['pitch_type'] == 'CH']
fs = data.loc[data['pitch_type'] == 'FS']
offspeed = ch.append(fs)
print('Off speed shape:', offspeed.shape)
rhp = data.loc[data['p_throws'] == 'R']
print('RHP shape:', rhp.shape)
lhp = data.loc[data['p_throws'] == 'L']
print('LHP shape:', lhp.shape)
rhp_rhh = data.loc[(data['p_throws'] == 'R') & (data['stand'] == 'R')]
print('RHP & RHH shape:', rhp_rhh.shape)
rhp_lhh = data.loc[(data['p_throws'] == 'R') & (data['stand'] == 'L')]
print('RHP & LHH shape:', rhp_lhh.shape)
lhp_rhh = data.loc[(data['p_throws'] == 'L') & (data['stand'] == 'R')]
print('LHP & RHH shape:', lhp_rhh.shape)
lhp_lhh = data.loc[(data['p_throws'] == 'L') & (data['stand'] == 'L')]
print('LHP & LHH shape:', lhp_lhh.shape)
rhp_fastball = fastball.loc[fastball['p_throws'] == 'R']
print('RHP Fastball shape:', rhp_fastball.shape)
lhp_fastball = fastball.loc[fastball['p_throws'] == 'L']
print('LHP Fastball shape:', lhp_fastball.shape)
rhp_breaking_ball = breaking_ball.loc[breaking_ball['p_throws'] == 'R']
print('RHP Breaking Ball shape:', rhp_breaking_ball.shape)
lhp_breaking_ball = breaking_ball.loc[breaking_ball['p_throws'] == 'L']
print('LHP Breaking Ball shape:', lhp_breaking_ball.shape)
rhp_offspeed = offspeed.loc[offspeed['p_throws'] == 'R']
print('RHP Offspeed shape:', rhp_offspeed.shape)
lhp_offspeed = offspeed.loc[offspeed['p_throws'] == 'L']
print('LHP Offspeed shape:', lhp_offspeed.shape)
zero_outs = data.loc[data['outs_when_up'] == 0]
print('0 outs:', zero_outs.shape)
one_out = data.loc[data['outs_when_up'] == 1]
print('1 out:', one_out.shape)
two_outs = data.loc[data['outs_when_up'] == 2]
print('2 outs:', two_outs.shape)

# Random Forest Modeling